# 🚀 Final Training Notebook for Multilingual Hate Detection (XLM-RoBERTa Base)
This notebook is optimized for Google Colab (T4 GPU recommended).
It uses the **FINAL** dataset (`Final_Dataset.csv`) which includes:
- 90,710 deduplicated samples (54k train, 36k val/test)
- Zero data leakage between splits
- Balanced class distributions across all tasks
- Cleaned text with artifacts removed

**Model:** XLM-RoBERTa Base (270M params) - Optimal for dataset size

**Steps:**
1. Mount Drive.
2. Install Dependencies.

3. Load Data.5. Save Checkpoints.
4. Train with Mixed Precision (AMP).

In [ ]:
# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create checkpoint directory
import os
CHECKPOINT_DIR = '/content/drive/MyDrive/thesis/checkpoints/'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
print(f"✅ Checkpoints will be saved to: {CHECKPOINT_DIR}")

In [ ]:
# 2. Install Dependencies & Check GPU
# Force upgrade to fix 'GenerationMixin' error and ensure compatibility
!pip install -U transformers accelerate wandb scikit-learn seaborn matplotlib

import torch
import os
import sys

if torch.cuda.is_available():
    print(f"✅ GPU Available: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    # 🚀 Performance Optimization
    torch.backends.cudnn.benchmark = True
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
else:
    print("❌ No GPU detected. Please enable GPU in Runtime > Change runtime type.")

In [ ]:
# 2.1 Pre-download Model (Turbo Mode 🚀)
import os

print("⏳ Downloading XLM-RoBERTa Base (High Speed)...")

# 1. Install hf_transfer (Rust-based accelerator)
!pip install -q huggingface_hub hf_transfer

# 2. Enable the accelerator
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

# 3. Download
# Note: If this fails with "command not found", restart the runtime once.
!huggingface-cli download xlm-roberta-base --local-dir ./xlm-roberta-base --exclude "*.msgpack" "*.h5" "*.ot"

print("✅ Model downloaded to ./xlm-roberta-base")

In [ ]:
# 3. Load Dataset
import pandas as pd
import numpy as np

# Path to the uploaded FINAL dataset
DATASET_PATH = '/content/drive/MyDrive/thesis/dataset/Final_Dataset.csv'

if not os.path.exists(DATASET_PATH):
    print(f"❌ Error: Dataset not found at {DATASET_PATH}")
    print("Please upload 'Final_Dataset.csv' to 'thesis/dataset/' in your Drive.")
else:
    df = pd.read_csv(DATASET_PATH)
    print(f"✅ Loaded dataset: {len(df)} rows")
    print("\nSplit distribution:")
    print(df['split'].value_counts())
    print("\nLanguage distribution:")
    print(df['language'].value_counts())
    print("\nSeverity distribution:")
    print(df['severity'].value_counts().sort_index())
    print("\nHate Type distribution:")
    print(df['hate_type'].value_counts().sort_index())

In [ ]:
# 4. Define Dataset Class
from torch.utils.data import Dataset, DataLoader
from transformers import XLMRobertaTokenizer

class HateDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=160):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        text = str(row['text'])
        
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        # Extract labels (use -1 for missing)
        hate_type = int(row['hate_type'])
        target_group = int(row['target_group'])
        severity = int(row['severity'])
        
        # Create masks: True if label is valid (not -1)
        hate_type_mask = hate_type != -1
        target_group_mask = target_group != -1
        severity_mask = severity != -1
        
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'hate_type': torch.tensor(max(0, hate_type), dtype=torch.long),
            'target_group': torch.tensor(max(0, target_group), dtype=torch.long),
            'severity': torch.tensor(max(0, severity), dtype=torch.long),
            'hate_type_mask': torch.tensor(hate_type_mask, dtype=torch.bool),
            'target_group_mask': torch.tensor(target_group_mask, dtype=torch.bool),
            'severity_mask': torch.tensor(severity_mask, dtype=torch.bool),
        }

# Initialize Tokenizer
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
print("✅ Tokenizer loaded")

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch

class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none', weight=self.alpha)
        pt = torch.exp(-ce_loss)
        focal_loss = (1 - pt) ** self.gamma * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

# 🔥 OPTIMIZED WEIGHTS FOR BALANCED PERFORMANCE
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1. Hate Type Weights (6 Classes)
# Balanced to handle the 3:1 ratio of Not-Hate vs Hate
ht_weights = torch.tensor([1.0, 2.5, 2.5, 2.5, 2.0, 2.5], device=device, dtype=torch.float)

# 2. Target Group Weights (4 Classes)
# Community is dominant, so we boost Individual/Org
tg_weights = torch.tensor([1.0, 3.0, 3.0, 1.0], device=device, dtype=torch.float) 

# 3. Severity Weights (4 Classes) - CRITICAL FIX 🛠️
# Previous issue: Model predicted "High" for everything.
# Fix: Equalize weights for Low(1), Medium(2), High(3) to force feature learning.
# We boost all hate severities to 3.0 to prioritize them over None(0).
sv_weights = torch.tensor([1.0, 3.0, 3.0, 3.0], device=device, dtype=torch.float)

# Initialize Weighted Focal Loss
loss_fn_ht = FocalLoss(alpha=ht_weights, gamma=2.0)
loss_fn_tg = FocalLoss(alpha=tg_weights, gamma=2.0)
loss_fn_sv = FocalLoss(alpha=sv_weights, gamma=2.0)

def multitask_loss(hate_type_logits, target_group_logits, severity_logits,
                   targets, masks, task_weights=(1.0, 1.0, 1.5)): 
    # ⬆️ NOTE: Severity task weight boosted to 1.5 to prioritize learning it
    
    total_loss = 0.0
    
    # Hate type loss
    ht_mask = masks['hate_type'].bool()
    if ht_mask.any():
        loss_ht = loss_fn_ht(hate_type_logits[ht_mask], targets['hate_type'][ht_mask])
        total_loss += task_weights[0] * loss_ht
        
    # Target group loss
    tg_mask = masks['target_group'].bool()
    if tg_mask.any():
        loss_tg = loss_fn_tg(target_group_logits[tg_mask], targets['target_group'][tg_mask])
        total_loss += task_weights[1] * loss_tg
        
    # Severity loss
    sv_mask = masks['severity'].bool()
    if sv_mask.any():
        loss_sv = loss_fn_sv(severity_logits[sv_mask], targets['severity'][sv_mask])
        total_loss += task_weights[2] * loss_sv
        
    return total_loss

In [ ]:
# 5. Define Model Architecture
import torch.nn as nn
import os
from transformers import XLMRobertaModel

class MultiTaskXLMRRoberta(nn.Module):
    def __init__(self, model_name='xlm-roberta-base', dropout=0.2):
        super(MultiTaskXLMRRoberta, self).__init__()
        
        # Check if local cache exists (from CLI download)
        if model_name == 'xlm-roberta-base' and os.path.exists('./xlm-roberta-base'):
            print(f"📂 Loading model from local cache: ./xlm-roberta-base")
            model_name = './xlm-roberta-base'
            
        self.backbone = XLMRobertaModel.from_pretrained(model_name)
        self.hidden_size = self.backbone.config.hidden_size
        self.dropout = nn.Dropout(dropout)
        
        # Classification Heads
        self.hate_type_head = nn.Linear(self.hidden_size, 6)    # 0-5
        self.target_group_head = nn.Linear(self.hidden_size, 4) # 0-3
        self.severity_head = nn.Linear(self.hidden_size, 4)     # 0-3
        
    def forward(self, input_ids, attention_mask):
        outputs = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
        # Use CLS token representation (first token)
        pooled_output = outputs.last_hidden_state[:, 0, :]
        pooled_output = self.dropout(pooled_output)
        
        hate_type_logits = self.hate_type_head(pooled_output)
        target_group_logits = self.target_group_head(pooled_output)
        severity_logits = self.severity_head(pooled_output)
        
        return hate_type_logits, target_group_logits, severity_logits

print("✅ MultiTaskXLMRRoberta model defined")

In [ ]:
# 7. Training Loop with Smart LR Adjustment
# We need to lower LR when unfreezing to prevent divergence

def train_model(model, train_loader, val_loader, total_epochs=8):
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, weight_decay=1e-2)
    
    # Scheduler: Linear warmup then decay
    total_steps = len(train_loader) * total_epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps=int(0.1 * total_steps), 
                                                num_training_steps=total_steps)
    
    scaler = amp.GradScaler()
    
    best_val_loss = float('inf')
    
    # Checkpoint names
    last_ckpt_path = os.path.join(CHECKPOINT_DIR, 'xlmr_base_final_last.pt')
    best_ckpt_path = os.path.join(CHECKPOINT_DIR, 'xlmr_base_final_best.pt')
    
    start_epoch = 0
    
    print(f"🚀 Starting SMART Training Strategy")
    print(f"   Phase 1 (Epoch 1-2): Backbone Frozen (Heads only)")
    print(f"   Phase 2 (Epoch 3+):  Full Fine-Tuning (Lower LR)")
    
    for epoch in range(start_epoch, total_epochs):
        start_time = time.time()
        print(f"\nEpoch {epoch+1}/{total_epochs}")
        
        # SMART FREEZING & LR LOGIC
        if epoch < 2:
            freeze_backbone(model, freeze=True)
            # Standard LR for heads
            for param_group in optimizer.param_groups:
                param_group['lr'] = 2e-5
        else:
            freeze_backbone(model, freeze=False)
            # 🔥 CRITICAL FIX: Lower LR by 10x when unfreezing backbone
            # This prevents the "Epoch 4 Divergence" we saw earlier
            for param_group in optimizer.param_groups:
                param_group['lr'] = 2e-6 
            
        model.train()
        total_loss = 0
        
        loop = tqdm(train_loader, leave=True)
        for batch in loop:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            
            targets = {k: batch[k].to(device) for k in ['hate_type', 'target_group', 'severity']}
            masks = {k: batch[f'{k}_mask'].to(device) for k in ['hate_type', 'target_group', 'severity']}
            
            optimizer.zero_grad()
            
            with amp.autocast():
                ht_logits, tg_logits, sv_logits = model(input_ids, attention_mask)
                loss = multitask_loss(ht_logits, tg_logits, sv_logits, targets, masks)
            
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(optimizer)
            scaler.update()
            # scheduler.step() # Disable scheduler since we manually control LR
            
            total_loss += loss.item()
            loop.set_description(f"Loss: {loss.item():.4f}")
            
        avg_train_loss = total_loss / len(train_loader)
        print(f"Average Train Loss: {avg_train_loss:.4f}")
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                targets = {k: batch[k].to(device) for k in ['hate_type', 'target_group', 'severity']}
                masks = {k: batch[f'{k}_mask'].to(device) for k in ['hate_type', 'target_group', 'severity']}
                
                ht_logits, tg_logits, sv_logits = model(input_ids, attention_mask)
                loss = multitask_loss(ht_logits, tg_logits, sv_logits, targets, masks)
                val_loss += loss.item()
                
        avg_val_loss = val_loss / len(val_loader)
        epoch_time = (time.time() - start_time) / 60
        print(f"Validation Loss: {avg_val_loss:.4f} | Time: {epoch_time:.1f} min")
        
        # Save Best Checkpoint
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), best_ckpt_path)
            print(f"🔥 New Best Model Saved: {best_ckpt_path}")
            
        # Save Last Checkpoint
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_val_loss': best_val_loss
        }, last_ckpt_path)

In [ ]:
# 7. Prepare Data Loaders
from torch.utils.data import DataLoader
import torch

# Split Data
train_df = df[df['split'] == 'train']
val_df = df[df['split'] == 'val']

train_dataset = HateDataset(train_df, tokenizer)
val_dataset = HateDataset(val_df, tokenizer)

# 🚀 BATCH SIZE CONFIGURATION
# User requested fixed batch size of 16
BATCH_SIZE = 16
print(f"🚀 Using Fixed Batch Size: {BATCH_SIZE}")

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")

In [ ]:
# 8. Training Loop (Smart Layer-Wise Learning)
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm
import torch.cuda.amp as amp
import time
import os
import torch

# 🛠️ Memory Optimization
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

def freeze_backbone(model, freeze=True):
    for param in model.backbone.parameters():
        param.requires_grad = not freeze
    status = "❄️ FROZEN" if freeze else "🔥 UNFROZEN"
    print(f"Model Backbone is now {status}")

def evaluate(model, loader, device):
    model.eval()
    total_loss = 0
    
    all_preds = {'hate_type': [], 'target_group': [], 'severity': []}
    all_labels = {'hate_type': [], 'target_group': [], 'severity': []}
    
    with torch.no_grad():
        for batch in loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            
            targets = {k: batch[k].to(device) for k in ['hate_type', 'target_group', 'severity']}
            
            # Fix masks for validation
            masks = {
                'hate_type': batch['hate_type_mask'].to(device),
                'target_group': batch['target_group_mask'].to(device),
                'severity': batch['severity_mask'].to(device)
            }
            
            ht_logits, tg_logits, sv_logits = model(input_ids, attention_mask)
            loss = multitask_loss(ht_logits, tg_logits, sv_logits, targets, masks)
            total_loss += loss.item()
            
            # Collect preds for F1
            for task, logits in zip(['hate_type', 'target_group', 'severity'], [ht_logits, tg_logits, sv_logits]):
                preds = torch.argmax(logits, dim=1)
                mask = masks[task].bool()
                
                if mask.any():
                    all_preds[task].extend(preds[mask].cpu().numpy())
                    all_labels[task].extend(targets[task][mask].cpu().numpy())
                    
    avg_loss = total_loss / len(loader)
    
    metrics = {}
    for task in ['hate_type', 'target_group', 'severity']:
        if len(all_labels[task]) > 0:
            metrics[task] = {
                'macro_f1': f1_score(all_labels[task], all_preds[task], average='macro'),
                'micro_f1': f1_score(all_labels[task], all_preds[task], average='micro')
            }
        else:
            metrics[task] = {'macro_f1': 0.0, 'micro_f1': 0.0}
        
    return avg_loss, metrics

def train_model(total_epochs=5, resume_from_epoch=None):
    torch.cuda.empty_cache() # 🧹 Clear GPU cache before starting
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = MultiTaskXLMRRoberta(dropout=0.2).to(device) 
    
    # 🛠️ Dynamic Gradient Accumulation
    # We aim for an Effective Batch Size of 16
    # If BATCH_SIZE is 16 (A100), Steps = 1
    # If BATCH_SIZE is 4 (T4), Steps = 4
    TARGET_EFFECTIVE_BATCH = 16
    ACCUMULATION_STEPS = max(1, TARGET_EFFECTIVE_BATCH // BATCH_SIZE)
    
    print(f"⚙️ Configuration:")
    print(f"   - Physical Batch Size: {BATCH_SIZE}")
    print(f"   - Accumulation Steps:  {ACCUMULATION_STEPS}")
    print(f"   - Effective Batch Size: {BATCH_SIZE * ACCUMULATION_STEPS}")
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
    
    # Adjust total steps for accumulation
    total_steps = (len(train_loader) // ACCUMULATION_STEPS) * total_epochs
    
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(0.1*total_steps), num_training_steps=total_steps)
    scaler = torch.amp.GradScaler('cuda') 
    
    start_epoch = 0
    best_val_loss = float('inf')
    
    # Checkpoint names
    last_ckpt_path = os.path.join(CHECKPOINT_DIR, 'xlmr_smart_last.pt')
    best_ckpt_path = os.path.join(CHECKPOINT_DIR, 'xlmr_smart_best.pt')
    
    # Resume logic
    if os.path.exists(last_ckpt_path):
        print(f"🔄 Found checkpoint at {last_ckpt_path}. Attempting to resume...")
        try:
            checkpoint = torch.load(last_ckpt_path)
            
            # CASE 1: Full Checkpoint (Ideal)
            if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
                model.load_state_dict(checkpoint['model_state_dict'])
                optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                if 'scheduler_state_dict' in checkpoint:
                    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
                if 'scaler_state_dict' in checkpoint:
                    scaler.load_state_dict(checkpoint['scaler_state_dict'])
                
                start_epoch = checkpoint['epoch'] + 1
                best_val_loss = checkpoint.get('best_val_loss', float('inf'))
                print(f"✅ Successfully resumed from Epoch {start_epoch+1}")
                
            # CASE 2: Weights Only (Fallback)
            else:
                print("⚠️ Checkpoint contains weights only (no optimizer state).")
                model.load_state_dict(checkpoint) # Try loading directly
                print("✅ Weights loaded.")
                
                if resume_from_epoch is not None:
                    start_epoch = resume_from_epoch
                    print(f"⏩ Forcing start from Epoch {start_epoch+1} (User specified)")
                else:
                    print("⚠️ No epoch info found. Starting from Epoch 1.")

        except Exception as e:
            print(f"❌ Error loading checkpoint: {e}")
            print("🚀 Starting FRESH Smart Mode training.")
    else:
        print("🚀 Starting FRESH Smart Mode training.")

    # Override if user specified
    if resume_from_epoch is not None and start_epoch == 0:
         start_epoch = resume_from_epoch
         print(f"⏩ Manual override: Starting from Epoch {start_epoch+1}")

    if start_epoch >= total_epochs:
        print("✅ Training already completed!")
        return
    
    print(f"🚀 Starting SMART Training Strategy (10 Epochs)")
    print(f"   Phase 1 (Epoch 1):   Backbone Frozen (Heads Warmup)")
    print(f"   Phase 2 (Epoch 2-10): Full Fine-Tuning")
    
    for epoch in range(start_epoch, total_epochs):
        start_time = time.time()
        print(f"\nEpoch {epoch+1}/{total_epochs}")
        
        # SMART FREEZING LOGIC
        if epoch < 1:
            freeze_backbone(model, freeze=True)
        else:
            freeze_backbone(model, freeze=False)
            
        model.train()
        optimizer.zero_grad() # Reset gradients
        total_loss = 0
        
        loop = tqdm(train_loader, leave=True)
        
        for i, batch in enumerate(loop):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            
            # Forward (Mixed Precision)
            with torch.amp.autocast('cuda'):
                ht_logits, tg_logits, sv_logits = model(input_ids, attention_mask)
                
                targets = {k: v.to(device) for k, v in batch.items() if k in ['hate_type', 'target_group', 'severity']}
                masks = {
                    'hate_type': batch['hate_type_mask'].to(device),
                    'target_group': batch['target_group_mask'].to(device),
                    'severity': batch['severity_mask'].to(device)
                }
                
                loss = multitask_loss(ht_logits, tg_logits, sv_logits, targets, masks)
                loss = loss / ACCUMULATION_STEPS # Normalize loss
            
            # Backward
            scaler.scale(loss).backward()
            
            # Step Optimizer every ACCUMULATION_STEPS
            if (i + 1) % ACCUMULATION_STEPS == 0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                scaler.step(optimizer)
                scaler.update()
                scheduler.step()
                optimizer.zero_grad()
            
            total_loss += loss.item() * ACCUMULATION_STEPS
            loop.set_description(f"Loss: {(loss.item() * ACCUMULATION_STEPS):.4f}")
            
        avg_train_loss = total_loss / len(train_loader)
        
        # Validation
        val_loss, metrics = evaluate(model, val_loader, device)
        
        print(f"📉 Train Loss: {avg_train_loss:.4f} | Val Loss: {val_loss:.4f}")
        print(f"🏆 Val Macro F1 - HT: {metrics['hate_type']['macro_f1']:.3f} | TG: {metrics['target_group']['macro_f1']:.3f} | SV: {metrics['severity']['macro_f1']:.3f}")
        
        # Save Best Model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), best_ckpt_path)
            print(f"🔥 New Best Model Saved: {best_ckpt_path}")
            
        # Save Last Checkpoint
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'scaler_state_dict': scaler.state_dict(),
            'best_val_loss': best_val_loss
        }, last_ckpt_path)

In [ ]:
# 🚀 START TRAINING
# We run for 10 epochs as requested to ensure deep learning on the balanced dataset.
train_model(total_epochs=10)

In [ ]:
# 9. Final Evaluation on Test Set (Smart Model)
from sklearn.metrics import classification_report
import torch
import os
from tqdm.notebook import tqdm

def evaluate_test_set():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    print("⏳ Loading best SMART model for evaluation...")
    model = MultiTaskXLMRRoberta(dropout=0.3) 
    # Load best checkpoint
    ckpt_path = os.path.join(CHECKPOINT_DIR, 'xlmr_smart_best.pt')
    if not os.path.exists(ckpt_path):
        print("❌ No smart checkpoint found. Run training first.")
        return
        
    model.load_state_dict(torch.load(ckpt_path))
    model.to(device)
    model.eval()
    
    # Prepare Test Loader
    test_df = df[df['split'] == 'test']
    test_dataset = HateDataset(test_df, tokenizer)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=2)
    
    print(f"🚀 Evaluating on {len(test_df)} test samples...")
    
    # Storage for predictions and labels
    task_metrics = {
        'hate_type': {'preds': [], 'labels': []},
        'target_group': {'preds': [], 'labels': []},
        'severity': {'preds': [], 'labels': []}
    }
    
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            
            ht_logits, tg_logits, sv_logits = model(input_ids, attention_mask)
            
            # Helper to collect valid predictions
            def collect(logits, task_name):
                preds = torch.argmax(logits, dim=1).cpu().numpy()
                labels = batch[task_name].cpu().numpy()
                masks = batch[f'{task_name}_mask'].cpu().numpy()
                
                for p, l, m in zip(preds, labels, masks):
                    if m: # Only keep valid labels
                        task_metrics[task_name]['preds'].append(p)
                        task_metrics[task_name]['labels'].append(l)

            collect(ht_logits, 'hate_type')
            collect(tg_logits, 'target_group')
            collect(sv_logits, 'severity')
            
    # Print Reports
    for task, data in task_metrics.items():
        print(f"\n📊 --- {task.upper().replace('_', ' ')} Report ---")
        if len(data['labels']) > 0:
            print(classification_report(data['labels'], data['preds'], digits=4))
        else:
            print("No valid labels found for this task in test set.")

evaluate_test_set()

In [ ]:
# 10. Visualize Confusion Matrices (Thesis Quality Plots)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np

def plot_confusion_matrices(model, loader, device):
    model.eval()
    all_preds = {'hate_type': [], 'target_group': [], 'severity': []}
    all_labels = {'hate_type': [], 'target_group': [], 'severity': []}
    
    with torch.no_grad():
        for batch in tqdm(loader, desc="Collecting Predictions"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            
            ht_logits, tg_logits, sv_logits = model(input_ids, attention_mask)
            
            # Helper to collect valid predictions
            def collect(logits, task_name):
                preds = torch.argmax(logits, dim=1).cpu().numpy()
                labels = batch[task_name].cpu().numpy()
                masks = batch[f'{task_name}_mask'].cpu().numpy()
                
                for p, l, m in zip(preds, labels, masks):
                    if m: # Only keep valid labels
                        all_preds[task_name].append(p)
                        all_labels[task_name].append(l)

            collect(ht_logits, 'hate_type')
            collect(tg_logits, 'target_group')
            collect(sv_logits, 'severity')

    # --- LABELS (Update these based on your specific dataset mapping) ---
    # Based on standard mappings:
    ht_labels = ['Not Hate', 'Political', 'Religious', 'Gender', 'Geopolitical', 'Personal'] 
    tg_labels = ['Other', 'Individual', 'Organization', 'Community']
    sv_labels = ['Level 0', 'Level 1', 'Level 2', 'Level 3']

    # Plotting
    fig, axes = plt.subplots(1, 3, figsize=(24, 8))
    
    # Task 1: Hate Type
    cm_ht = confusion_matrix(all_labels['hate_type'], all_preds['hate_type'])
    sns.heatmap(cm_ht, annot=True, fmt='d', cmap='Blues', ax=axes[0], 
                xticklabels=ht_labels, yticklabels=ht_labels)
    axes[0].set_title('Hate Type Confusion Matrix', fontsize=14)
    axes[0].set_xlabel('Predicted', fontsize=12)
    axes[0].set_ylabel('Actual', fontsize=12)
    axes[0].tick_params(axis='x', rotation=45)
    
    # Task 2: Target Group
    cm_tg = confusion_matrix(all_labels['target_group'], all_preds['target_group'])
    sns.heatmap(cm_tg, annot=True, fmt='d', cmap='Greens', ax=axes[1],
                xticklabels=tg_labels, yticklabels=tg_labels)
    axes[1].set_title('Target Group Confusion Matrix', fontsize=14)
    axes[1].set_xlabel('Predicted', fontsize=12)
    axes[1].set_ylabel('Actual', fontsize=12)
    axes[1].tick_params(axis='x', rotation=45)
    
    # Task 3: Severity
    cm_sv = confusion_matrix(all_labels['severity'], all_preds['severity'])
    sns.heatmap(cm_sv, annot=True, fmt='d', cmap='Oranges', ax=axes[2],
                xticklabels=sv_labels, yticklabels=sv_labels)
    axes[2].set_title('Severity Confusion Matrix', fontsize=14)
    axes[2].set_xlabel('Predicted', fontsize=12)
    axes[2].set_ylabel('Actual', fontsize=12)
    
    plt.tight_layout()
    
    # Save for Thesis
    save_path = os.path.join(CHECKPOINT_DIR, 'confusion_matrices.png')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"✅ Plot saved to: {save_path}")
    
    plt.show()

# Run Visualization
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MultiTaskXLMRRoberta(dropout=0.3)
model.load_state_dict(torch.load(os.path.join(CHECKPOINT_DIR, 'xlmr_smart_best.pt')))
model.to(device)

test_df = df[df['split'] == 'test']
test_dataset = HateDataset(test_df, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=2)

plot_confusion_matrices(model, test_loader, device)